# Search in crossref archive

Verify what are the ids stored in crossref when funder is included 
This could help build a query that looks for specific funder insted of retrieving all records with any funder

Could do the same for affiliations?
Could launch a search for the whole year?


In [1]:
# Connecting to the db
import lib.handle_db as dbh

# read and write csv files
import lib.handle_csv as csv_rw

# date functions
from datetime import datetime, date, timedelta

# managing files and file paths
from pathlib import Path

# import custom functions (common to various notebooks)
import processing_functions as pr_fns

#CR libraries
from crossref.restful import Works, Etiquette



# search for UKCH Awards in CR record
def award_in_crossref(aw, award_list):
    ukch_wks =[]
    not_revised = []
    for wk in aw:      
        awd_list = []
        if 'funder' in wk.keys():
            for fdr in wk['funder']:
                if 'award' in fdr.keys():
                   awds = 0
                   for awd in fdr['award']:
                        if awd in award_list:
                            awd_list.append(awd)
                            #print (fdr)
                            break
        else:
            not_revised.append(wk)
        if len(awd_list) > 0:
            ukch_wks.append(wk)
    return ukch_wks, not_revised


In [2]:
# set output vars
base_dir = './pub_search_crossref'
csv_file_out = 'cr_funder202311_b.csv'
out_file = Path(base_dir, csv_file_out)

# CR etiquette
my_etiquette = Etiquette('UK Catalysis Hub - Catalysis Data Infrastructure', 
                         'Prototype 1', 
                         'https://ukcatalysishub.co.uk/core/', 
                         'nieva@rc-harwell.ac.uk')

# cr api for retrieving works 
works = Works(etiquette=my_etiquette)

award_list = ['EP/R026939/1', 'EP/R026815/1', 'EP/R026645/1', 'EP/R027129/1', 'EP/M013219/1',
               'EP/K014706/2', 'EP/K014668/1', 'EP/K014854/1', 'EP/K014714/1',]


In [ ]:
# open DB
db_name = 'production'

# 1 currend app DB
ukchapp_db = "db_files/" + db_name + ".sqlite3"

# get publication data from the ukch app
app_pubs = pr_fns.get_pub_data(ukchapp_db)


csv_data = {}
for a_pub in app_pubs:
    if a_pub[0] > 633 :
        pub_id = a_pub[0]
        pub_title = a_pub[1]
        pub_doi = a_pub[2]
        pub_url = a_pub[3]
        pub_pdf = a_pub[4]

        db_record = {"pub_id": pub_id, "pub_title": pub_title, "pub_doi":pub_doi, "pub_url":pub_url, "pub_pdf": pub_pdf}
        funder_data = {'funder_doi':'','funder_name':'','funder_doi_asserted_by':'','funder_awards':[]}
        cr_dates={'indexed':'','created':'','deposited':''}
        if pub_doi != None or \
            pub_doi == '':
            print(works.filter(doi=pub_doi.strip()).url)
            cr_works = works.filter(doi=pub_doi.strip())

            for crwk in cr_works:     
                awd_list = []             
                if 'funder' in crwk.keys():
                    for fdr in crwk['funder']:
                        if 'award' in fdr.keys():
                           awds = 0
                           for awd in fdr['award']:
                                if awd in award_list:
                                    awd_list.append(awd)
                                    if funder_data['funder_name'] == "":
                                        if 'DOI' in fdr.keys(): funder_data['funder_doi'] = fdr['DOI']
                                        if 'name' in fdr.keys(): funder_data['funder_name'] = fdr['name']
                                        if 'doi-asserted-by' in fdr.keys(): funder_data['funder_doi_asserted_by'] = fdr['doi-asserted-by']
                        funder_data["funder_awards"] = awd_list
                cr_dates['indexed']=crwk['indexed']['date-time']
                cr_dates['created']=crwk['created']['date-time']
                cr_dates['deposited']=crwk['deposited']['date-time']
        db_record = {**db_record, **funder_data, **cr_dates}        
        csv_data[pub_id] =db_record
    

In [3]:
# WRITE TO FILE
if len(csv_data) > 0:
    csv_rw.write_csv_data(csv_data, out_file) 

NameError: name 'csv_data' is not defined

In [ ]:
csv_data

In [4]:
start_date = end_date = date(2023, 1, 1)

works.filter(from_created_date=str(start_date)).url

'https://api.crossref.org/works?filter=from-created-date%3A2023-11-01'

In [17]:
ukch_awards = ['EP/R026939/1', 'EP/R026815/1', 'EP/R026645/1', 'EP/R027129/1', 'EP/M013219/1',
               'EP/K014706/2', 'EP/K014668/1', 'EP/K014854/1', 'EP/K014714/1',]

mcc_awards =['EP/R029431','EP/P020194', 'EP/T022213', 'EP/D504872', 'EP/F067496',]

award_list = ukch_awards + mcc_awards

In [32]:
a_filter = ""

publications = {}
for this_aw in award_list:
    a_filter = works.filter(from_created_date=str(start_date)).filter(award__number=this_aw).url
    print(a_filter)
    pub_w_grant = works.filter(from_created_date=str(start_date)).filter(award__number=this_aw)
    
    for wk in pub_w_grant:
        if not wk['DOI'] in publications:
            publications[wk['DOI']] = wk
    
        awd_list = [] 
        for fdr in wk['funder']:
            if 'award' in fdr.keys():
               awds = 0
               for awd in fdr['award']:
                   if awd in award_list:
                       awd_list.append(awd)


https://api.crossref.org/works?filter=from-created-date%3A2023-11-01%2Caward.number%3AEP%2FR026939%2F1
https://api.crossref.org/works?filter=from-created-date%3A2023-11-01%2Caward.number%3AEP%2FR026815%2F1
https://api.crossref.org/works?filter=from-created-date%3A2023-11-01%2Caward.number%3AEP%2FR026645%2F1
https://api.crossref.org/works?filter=from-created-date%3A2023-11-01%2Caward.number%3AEP%2FR027129%2F1
https://api.crossref.org/works?filter=from-created-date%3A2023-11-01%2Caward.number%3AEP%2FM013219%2F1
https://api.crossref.org/works?filter=from-created-date%3A2023-11-01%2Caward.number%3AEP%2FK014706%2F2
https://api.crossref.org/works?filter=from-created-date%3A2023-11-01%2Caward.number%3AEP%2FK014668%2F1
https://api.crossref.org/works?filter=from-created-date%3A2023-11-01%2Caward.number%3AEP%2FK014854%2F1
https://api.crossref.org/works?filter=from-created-date%3A2023-11-01%2Caward.number%3AEP%2FK014714%2F1
https://api.crossref.org/works?filter=from-created-date%3A2023-11-01%2Caw

In [33]:
def get_authors(wk):
    author_list = ""
    if 'author' in wk.keys() :
        for autr in wk['author']:
            if author_lists == "":
                author_list = autr['family'] + (", "+ autr ['given'] if 'given' in autr.keys() else "" )
            else:
                author_list += ", " + autr['family']+ (", "+ autr ['given'] if 'given' in autr.keys() else "" )
    return author_list

def get_awards(wk):
    mcc_pub = ukch_pub = False
    award_list = ""
    for fdr in wk['funder']:
        if 'award' in fdr.keys():
          for awd in fdr['award']:
               if awd in award_list:
                    if award_list  == "":
                        award_list = awd
                    else:
                        award_list += ", " +awd
                    if awd in ukch_awards:
                        ukch_pub = True
                    if awd in mcc_awards:
                        mcc_pub = True
    return award_list, mcc_pub, ukch_pub  

def get_pub_year(wk):
    ol_year = 0
    pr_year = 0
    pub_year = 0
    if 'published-online' in wk.keys() and 'date-parts' in wk['published-online'].keys():
        ol_year = int(wk['published-online']['date-parts'][0][0])
    if 'published-print' in wk.keys() and 'date-parts' in wk['published-print'].keys():
        pr_year = int(wk['published-print']['date-parts'][0][0])
    if pr_year > 0 and ol_year > 0:
        if pr_year > ol_year:
            pub_year = ol_year
        else:
            pub_year = pr_year
    elif ol_year > 0:
        pub_year = ol_year
    elif pr_year > 0:
        pub_year = pr_year
    return  pub_year

In [31]:
foud_pubs = {}
for wk_doi in publications:
    wk = publications[wk_doi]
    
    art_authors = get_authors(wk)
    found_award, mcc_pub, ukch_pub = get_awards(wk)
    pub_year = get_pub_year(wk)    
    print(art_authors,"|",pub_year,"|",wk['title'][0],
          "|", wk['DOI'],"|", fund_award)     
    this_pub = {'authors': art_authors,
                'year': pub_year,
                'title': wk['title'][0],
                'DOI': wk['DOI'],
                'awards': fund_award,
                'mcc': int(mcc_pub),
                'ukch': int(ukch_pub)}
    if not wk['DOI'] in foud_pubs:
         foud_pubs[wk['DOI']]= this_pub

NameError: name 'get_authors' is not defined

In [26]:
len(foud_pubs)

6

In [28]:
if len(foud_pubs) > 0:
    csv_rw.write_csv_data(foud_pubs, 'cr_check_'+str(end_date)+'a.csv') 
    foud_pubs = {}
